In [15]:
import pandas as pd
from io import StringIO
from html.parser import HTMLParser


In [4]:
df = pd.read_xml('tmp/Posts.xml', xpath='//posts/row')
df_backup = df.copy()

In [22]:
df = df_backup.copy()
df = df[(df["Score"] > 0) & (df["PostTypeId"] == 1)].sort_values(by="Score", ascending=False)

# body is html, so we need to strip the tags

class MLStripper(HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.text = StringIO()
    def handle_data(self, d):
        self.text.write(d)
    def get_data(self):
        return self.text.getvalue()

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

df["Body"] = df["Body"].apply(strip_tags)

# strip at start and end
df["Body"] = df["Body"].str.strip()

# add <SPECIAL:END> to end of each post
df["Body"] = df["Body"] + " <SPECIAL:END>"

# save body as posts.txt
with open("tmp/posts.txt", "w", encoding="utf-8") as f:
    for post in df["Body"]:
        f.write(post)